In [19]:
import pandas as pd

In [20]:
out_bw = pd.read_csv('/public/xye6/Xerox/BW_outscope_withppm.csv')
del out_bw['dim_asset_inscope']
del out_bw['p-value']

In [21]:
out_color = pd.read_csv('/public/xye6/Xerox/CL_outscope_withppm.csv')
del out_color['dim_asset_inscope']
del out_color['p-value']

In [22]:
def find_bw_opp(data, ppm, percent, target):
    temp = data[data['max_ppm']<=ppm]
    group_data = temp.groupby(['dim_acct_sk','region','dim_model_manf_nm'])['BlackVolume'].sum()
    group_data = group_data.reset_index()
    temp_slope_data = data[['dim_acct_sk','region','dim_model_manf_nm','slopes']]
    group_data = pd.merge(group_data, temp_slope_data, on=['dim_acct_sk','region','dim_model_manf_nm'])
    group_data = group_data.sort_values('slopes',ascending=False)
    group_data.drop_duplicates(inplace=True)
    
    temp_sum = 0
    client_list = []
    acquire = []
    for i in range(len(group_data)):
        if temp_sum >= target:
            break
        result = round(group_data.iloc[i]['BlackVolume'] * percent)
        if result > 0:
            temp_sum += result
            client_list.append(i)
            acquire.append(result)
            
    if temp_sum < target:
        print('can''t meet the requirement, please set up a higher threshold for each clients or decrease target number')
    final_result = group_data.iloc[client_list]
    final_result['acquired_quantity'] = acquire
    
    return final_result

In [23]:
def find_color_opp(data, ppm, percent, target):
    temp = data[data['max_ppm']<=ppm]
    group_data = temp.groupby(['dim_acct_sk','region','dim_model_manf_nm'])['ColorVolume'].sum()
    group_data = group_data.reset_index()
    temp_slope_data = data[['dim_acct_sk','region','dim_model_manf_nm','slopes']]
    group_data = pd.merge(group_data, temp_slope_data, on=['dim_acct_sk','region','dim_model_manf_nm'])
    group_data = group_data.sort_values('slopes',ascending=False)
    group_data.drop_duplicates(inplace=True)
    
    temp_sum = 0
    client_list = []
    acquire = []
    for i in range(len(group_data)):
        if temp_sum >= target:
            break
        result = round(group_data.iloc[i]['ColorVolume'] * percent)
        if result > 0:
            temp_sum += result
            client_list.append(i)
            acquire.append(result)
            
    if temp_sum < target:
        print('can''t meet the requirement, please set up a higher threshold for each clients or decrease target number')
    final_result = group_data.iloc[client_list]
    final_result['acquired_quantity'] = acquire
    
    return final_result

## generate client list for black volume
### change the parameters in next two lines (data table, ppm, percent, target)
#### check the report/readme to know details of parameters

In [24]:
black_target = 2 * 10 ** 9

In [25]:
bw_list = find_bw_opp(out_bw, 150, 0.3, black_target)

/software/anaconda3/5.2.0/lib/python3.6/site-packages/ipykernel/__main__.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


## generate client list for color volume
### change the parameters in next two lines (data table, ppm, percent, target)
#### check the report/readme to know details of parameters

In [26]:
color_target = 10 ** 8

In [27]:
color_list = find_color_opp(out_color, 150, 0.3, color_target)

cant meet the requirement, please set up a higher threshold for each clients or decrease target number


/software/anaconda3/5.2.0/lib/python3.6/site-packages/ipykernel/__main__.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [28]:
bw_list.to_csv('client_list_bw.csv',index=False)
color_list.to_csv('client_list_color.csv',index=False)